In [1]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization, Input
from tensorflow.keras.layers import Activation, AveragePooling2D, Concatenate, Add, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks  import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import os
import warnings
warnings.filterwarnings('ignore')

2025-02-11 15:06:42.517617: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 15:06:42.534911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739286402.569609   14911 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739286402.577387   14911 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 15:06:42.603307: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Define dataset paths
dataset_path = "/teamspace/studios/this_studio/.cache/kagglehub/datasets/mohamedhanyyy/chest-ctscan-images/versions/1/Data"

# Load datasets
img_size = (224, 224)
batch_size = 32

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "/train", 
    image_size=img_size, 
    batch_size=batch_size, 
    label_mode='categorical')

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "/valid", 
    image_size=img_size, 
    batch_size=batch_size, 
    label_mode='categorical')

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "/test", 
    image_size=img_size, 
    batch_size=batch_size, 
    label_mode='categorical')

Found 613 files belonging to 4 classes.
Found 72 files belonging to 4 classes.


2025-02-11 15:06:46.251115: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Found 315 files belonging to 4 classes.


In [3]:
from tensorflow.keras import layers, models
# Normalize images
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

In [4]:
from tensorflow.keras.applications import EfficientNetB0

# Load EfficientNetB0 without the top layer
base_model = EfficientNetB0(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3)
)

# Freeze the base model to keep pretrained weights
base_model.trainable = False


In [6]:
# Define number of classes
num_classes = 4

# Create new model on top of base
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Flatten features
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Output layer
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,055 (16.08 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [7]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # or categorical_crossentropy if one-hot encoded
    metrics=['accuracy']
)

In [8]:
history = model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=10
)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.2628 - loss: 1.4410 - val_accuracy: 0.3194 - val_loss: 1.3793
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.2814 - loss: 1.4000 - val_accuracy: 0.3194 - val_loss: 1.3886
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.2965 - loss: 1.3748 - val_accuracy: 0.3194 - val_loss: 1.3852
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 994ms/step - accuracy: 0.2705 - loss: 1.3885 - val_accuracy: 0.3194 - val_loss: 1.3986
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.2857 - loss: 1.3749 - val_accuracy: 0.3194 - val_loss: 1.3979
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.3106 - loss: 1.3792 - val_accuracy: 0.3194 - val_loss: 1.3901
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.3064 - loss: 1.3750 - val_accuracy: 0.3194 - val_loss: 1.3931
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.2882 - loss: 1.3768 - val_accuracy: 0.3194 - val_lo

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

In [9]:
base_model.trainable = True  # Unfreeze model

# Lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=5
)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 156s 5s/step - accuracy: 0.2163 - loss: 2.4352 - val_accuracy: 0.3194 - val_loss: 1.3901
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 93s 5s/step - accuracy: 0.1953 - loss: 2.1319 - val_accuracy: 0.3194 - val_loss: 1.3879
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 102s 5s/step - accuracy: 0.2458 - loss: 1.9980 - val_accuracy: 0.3194 - val_loss: 1.3830
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 99s 5s/step - accuracy: 0.2751 - loss: 1.7370 - val_accuracy: 0.3194 - val_loss: 1.3852
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 97s 5s/step - accuracy: 0.3452 - loss: 1.5519 - val_accuracy: 0.3194 - val_loss: 1.3827
